In [1]:
import sys
import os
import sklearn
import pandas as pd
import numpy as np
import xml
import re
from pprint import pprint

print(sys.version)

3.5.5 |Anaconda, Inc.| (default, May 13 2018, 21:12:35) 
[GCC 7.2.0]


In [2]:
# Load dataset from csv
data_dir_path = os.path.join('.', 'data')
job_ads_filename = 'stillingsannonser.csv'
job_ads_desc_filename = 'stillingsannonser-beskrivelser.csv'

df_ads = pd.read_csv(
    os.path.join(data_dir_path, job_ads_filename),
    sep=';'
)

df_ads_desc = pd.read_csv(
    os.path.join(data_dir_path, job_ads_desc_filename),
    sep=';'
)

# Merge datasets into one
df = pd.merge(df_ads, df_ads_desc)

df.head()

,stillingsnummer,nav_enhet_kode,registrert_dato,sistepubl_dato,statistikk_aar_mnd,offisiell_statistikk_flagg,stilling_kilde,arbeidssted_fylkesnummer,arbeidssted_fylke,arbeidssted_kommunenummer,...,yrke_grovgruppe,yrkeskode,yrke,yrkesbetegnelse,virksomhet_organisasjonsnr,virksomhet_navn,antall_stillinger,stillingstittel,statistikkperiode,stillingsbeskrivelse
0,101201612000013,101,12.12.2016,15.01.2017,201701,1,Reg av arb.giver på nav.no,01,Østfold,0101,...,Undervisning,2342,Førskolelærere,Barnehagelærer,974122545,YNGLINGEN BARNEHAGE,1,Pedagogisk leder,201701.0,<p>Pedagogisk leder i 73% stilling</p>
1,101201612000015,101,14.12.2016,28.12.2016,201701,1,Overført fra arbeidsgiver,01,Østfold,0101,...,Undervisning,2351,Spesialister i pedagogikk,Studiekonsulent,973967592,HØGSKOLEN I ØSTFOLD,1,Rådgiver - studentopptak og internasjonalisering,201701.0,Jobbnorge ID: 131633<br><br><p># i Østfold har...
2,101201612000016,101,14.12.2016,31.01.2017,201701,1,Overført fra arbeidsgiver,01,Østfold,0101,...,Undervisning,2310,Universitets- og høyskolelektorer/-lærere,Førsteamanuensis,973967592,HØGSKOLEN I ØSTFOLD,1,Undervisnings- og forskerstilling i musikk,201701.0,Jobbnorge ID: 131849<br><br><p>Vi søker en dyk...
3,101201612000017,101,15.12.2016,20.01.2017,201701,1,Reg av arb.giver på nav.no,01,Østfold,0101,...,Reiseliv og transport,9412,Kjøkkenassistenter,Anretningsassistent,973070452,AXXE AS,1,Kantinemedarbeider,201701.0,<p>Vi søker en medarbeider i 80% vikariat til...
4,101201612000018,101,15.12.2016,15.01.2017,201701,1,Overført fra arbeidsgiver,01,Østfold,0101,...,Undervisning,2310,Universitets- og høyskolelektorer/-lærere,Lærer (universitet og høyskole),973967592,HØGSKOLEN I ØSTFOLD,1,Undervisnings- og forskerstilling i kroppsøving,201701.0,Jobbnorge ID: 132043<br><br><p>Vi søker en dyk...


In [3]:
def remove_tags(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

# Select subset of columns
col = [
    'stilling_kilde', 
    'stillingsnummer', 
    'stillingsbeskrivelse', 
    'yrke_grovgruppe',
    'arbeidssted_fylkesnummer',
    'arbeidssted_kommunenummer',
    'virksomhet_navn',
]

df2 = df[col]
# Remove html tags from description
df2[['stillingsbeskrivelse']] = df['stillingsbeskrivelse'].map(remove_tags)

print(df2.shape)
df2.head()




(148144, 7)


/opt/conda/lib/python3.5/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.5/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,stilling_kilde,stillingsnummer,stillingsbeskrivelse,yrke_grovgruppe,arbeidssted_fylkesnummer,arbeidssted_kommunenummer,virksomhet_navn
0,Reg av arb.giver på nav.no,101201612000013,Pedagogisk leder i 73% stilling,Undervisning,01,0101,YNGLINGEN BARNEHAGE
1,Overført fra arbeidsgiver,101201612000015,Jobbnorge ID: 131633# i Østfold har vi ledig e...,Undervisning,01,0101,HØGSKOLEN I ØSTFOLD
2,Overført fra arbeidsgiver,101201612000016,Jobbnorge ID: 131849Vi søker en dyktig medarbe...,Undervisning,01,0101,HØGSKOLEN I ØSTFOLD
3,Reg av arb.giver på nav.no,101201612000017,Vi søker en medarbeider i 80% vikariat til de...,Reiseliv og transport,01,0101,AXXE AS
4,Overført fra arbeidsgiver,101201612000018,Jobbnorge ID: 132043Vi søker en dyktig medarbe...,Undervisning,01,0101,HØGSKOLEN I ØSTFOLD


In [63]:

X_train, X_test, y_train, y_test = train_test_split(
    df2[:10000], 
    df2['yrke_grovgruppe'][:10000],
    test_size=.5,
    random_state=305
)

X_train

,stilling_kilde,stillingsnummer,stillingsbeskrivelse,yrke_grovgruppe,arbeidssted_fylkesnummer,arbeidssted_kommunenummer,virksomhet_navn
8958,Annonsert i media,217201708000061,Det er en grunn til at det er akkurat du som ø...,Reiseliv og transport,02,0217,HOTELLDRIFT Q ENTRY AS
97180,Annonsert i media,1209201612000287,FMCG - Branding - # & #\r\n\r\n Analytisk &amp...,Ledere,12,1201,KING OSCAR AS
46396,Annonsert i media,331201708000188,"Novelda AS is a privately held, R&amp;D driven...",Meglere og konsulenter,03,0301,NOVELDA AS
27216,Reg av arb.giver på nav.no,313201709000162,Nøkkelkvalifikasjoner til søker:\r\n \r\n-Fagb...,Reiseliv og transport,03,0301,GASTRONOMIA AS
106899,Annonsert i media,1260201706000003,"Kort om stillinga\r\nLedig fast, nyoppretta st...",Akademiske yrker,12,1260,RADØY KOMMUNE PLAN- OG
49725,Overført fra arbeidsgiver,335201706000151,\r\nVi trenger deg som er interessert I en tra...,Akademiske yrker,03,0301,SYKEHUSPARTNER
33114,Annonsert i media,316201703000096,Barnemedisinsk sengepost 2 er en medisinsk sen...,"Helse, pleie og omsorg",03,0301,OSLO UNIVERSITETSSYKEHUS HF
95730,Overført fra arbeidsgiver,1206201711000007,"Er du en sulten selger, med gode dokumenterte ...",Butikk- og salgsarbeid,12,1201,*
47549,Reg av arb.giver på nav.no,334201701000073,# søker avlaster til en 4 år gammel gutt\r\n \...,Barne- og ungdomsarbeid,03,0301,AVLASTNINGSTJENESTEN
42768,Reg av arb.giver på nav.no,328201612000021,Vi søker etter en positiv og motivert person s...,Industriarbeid,03,0301,CS GROUP


In [64]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import LabelEncoder

import category_encoders as ce



class ItemSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, x, y=None):
        return self
    
    def transform(self, x):
        return x[self.column]

class DataFrameToArrayTransformer(BaseEstimator, TransformerMixin):    
    def fit(self, x, y=None):
        return self

    def transform(self, X):
        return X.values
    
    
class WordCountVectorizer(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self
    
    def transform(self, X):
        #return np.array(X.map(lambda x: len(x.split(' '))), dtype=pd.Series)
        #return np.transpose(X.map(lambda x: len(x.split(' '))).as_matrix())
        return X.map(lambda x: len(x.split(' '))).values.reshape(-1, 1)

text_clf = Pipeline([
    ('vect', CountVectorizer(max_df=.2)),
    ('tfidf', TfidfTransformer()),
    ('clf', LinearSVC()),
])


pipeline = Pipeline([
    # Use FeatureUnion to combine the features from subject and body
    ('union', FeatureUnion(
        transformer_list=[
            # Pipeline for pulling features from the post's subject line
            ('text', Pipeline([
                ('selector', ItemSelector('stillingsbeskrivelse')),
                ('counts', CountVectorizer()),
                ('tf_idf', TfidfTransformer())
            ])),
            ('company', Pipeline([
                ('selector', ItemSelector('virksomhet_navn')),
                ('counts', CountVectorizer()),
            ])),
            ('source', Pipeline([
                ('selector', ItemSelector('stilling_kilde')),
                ('array_transformer', DataFrameToArrayTransformer()),
                ('label', ce.BinaryEncoder())
            ])),
            ('kommune', Pipeline([
                ('selector', ItemSelector('arbeidssted_kommunenummer')),
                ('array_transformer', DataFrameToArrayTransformer()),
                ('label', ce.BinaryEncoder())
            ])),
            ('fylke', Pipeline([
                ('selector', ItemSelector('arbeidssted_fylkesnummer')),
                ('array_transformer', DataFrameToArrayTransformer()),
                ('label', ce.BinaryEncoder())
            ])),
            #('word_count', Pipeline([
            #    ('selector', ItemSelector('stillingsbeskrivelse')),
            #    ('count', WordCountVectorizer()),
            #])),
        ],
        transformer_weights = {
            'text': 1.0,
            'company': .2,
            'source': .2,
            'fylke': .2,
            'kommune': .2,
        },
    )),

    # Use a SVC classifier on the combined features
    ('svc', LinearSVC()),
])

pipeline = pipeline.fit(X_train, y_train)

In [65]:
predicted = pipeline.predict(X_test)
pprint(predicted)
np.mean(predicted == y_test)

array(['Helse, pleie og omsorg', 'Helse, pleie og omsorg', 'Undervisning',
       ..., 'Butikk- og salgsarbeid', 'Helse, pleie og omsorg',
       'Helse, pleie og omsorg'], dtype=object)


0.8236445620477373

In [66]:
from sklearn import metrics
print(metrics.classification_report(y_test, predicted))
print(metrics.accuracy_score(y_test, predicted))

                                    precision    recall  f1-score   support

                  Akademiske yrker       0.74      0.72      0.73      5103
           Barne- og ungdomsarbeid       0.82      0.74      0.78      2431
            Butikk- og salgsarbeid       0.84      0.88      0.86      5926
                    Bygg og anlegg       0.84      0.86      0.85      4806
            Helse, pleie og omsorg       0.91      0.95      0.93     18828
                    Industriarbeid       0.75      0.71      0.73      2980
Ingen yrkesbakgrunn eller uoppgitt       0.42      0.16      0.23       192
              Ingeniør- og ikt-fag       0.76      0.82      0.79      7089
       Jordbruk, skogbruk og fiske       0.73      0.57      0.64       431
                      Kontorarbeid       0.69      0.63      0.66      2998
                            Ledere       0.57      0.47      0.52      1878
            Meglere og konsulenter       0.64      0.63      0.64      3110
           

In [ ]:
from sklearn.model_selection import GridSearchCV
from time import time
parameters = {
    'vect__max_df': (0.25, 0.5, 0.75, 1.0),
    #'vect__max_features': (None, 5000, 10000, 50000),
    #'vect__ngram_range': ((1, 1), (1, 2)),  # unigrams or bigrams
    'tfidf__use_idf': (True, False),
    'tfidf__norm': ('l1', 'l2'),
    #'clf__alpha': (0.00001, 0.000001),
    #'clf__penalty': ('l2', 'elasticnet'),
    #'clf__n_iter': (10, 50, 80),
}

if __name__ == "__main__":
    # multiprocessing requires the fork to happen in a __main__ protected
    # block

    # find the best parameters for both the feature extraction and the
    # classifier
    grid_search = GridSearchCV(text_clf, parameters, n_jobs=-1, verbose=1)

    print("Performing grid search...")
    print("pipeline:", [name for name, _ in text_clf.steps])
    print("parameters:")
    pprint(parameters)
    t0 = time()
    grid_search.fit(X_test, y_test)
    print("done in %0.3fs" % (time() - t0))
    print()

    print("Best score: %0.3f" % grid_search.best_score_)
    print("Best parameters set:")
    best_parameters = grid_search.best_estimator_.get_params()
    for param_name in sorted(parameters.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))

In [ ]:
prediction = text_clf.predict(['er du en serviceinnstilt solstråle med sansen for kundebehandling?'])
prediction